In [ ]:
!pip install transformers
!pip install datasets
from google.colab import drive
drive.mount('/content/drive')
from transformers import AutoModelForSequenceClassification
from tqdm import tqdm
import torch
import numpy as np
from datasets import load_dataset
from torch.utils.data import DataLoader

In [ ]:
device="cuda"
model_dir="drive/MyDrive/models/"
data_dir="drive/MyDrive/data/"
batch_size = 16

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased").to(device)
model.load_state_dict(torch.load(model_dir+"classifier"))

In [ ]:
test_dataset=load_dataset('json', data_files=data_dir+"res_base_uncased.json")["train"]

In [ ]:
test_loader = DataLoader(
                test_dataset,
                batch_size=batch_size,
                #collate_fn=data_collator,
                num_workers=2,
                drop_last=False
            )

res_list = []
model.eval()
for idx, tokenized_examples in enumerate(tqdm(test_loader)):
  input_ids = torch.stack(tokenized_examples['input_ids'])
  attention_mask = torch.stack(tokenized_examples['attention_mask'])

  input_ids = torch.transpose(input_ids, 0, 1).to(device)
  attention_mask = torch.transpose(attention_mask, 0, 1).to(device)

  pred = torch.argmax(model(input_ids=input_ids,
          attention_mask=attention_mask)["logits"], dim=1).cpu().numpy()
  res_list.append(pred)

In [ ]:
res = np.concatenate(res_list, axis=0)
print(len(res))

In [ ]:
import pandas as pd
for i in range(len(res)):
  res[i] = -1 if res[i]==0 or res[i]==-1 else 1
data = {
    "Id": list(range(1, len(res)+1)),
    "Prediction":res   
}

data_df = pd.DataFrame(data, columns=["Id", "Prediction"])
data_df.to_csv(data_dir+"results.csv", index=False)